In [1]:
!pip install  openai langchain chromadb unstructured -q


[notice] A new release of pip available: 22.3.1 -> 23.2.1
[notice] To update, run: pip install --upgrade pip


In [1]:
from langchain.document_loaders.csv_loader import CSVLoader

def load_csvs(directory):
    loader = CSVLoader(directory)
    csv_data = loader.load()
    return csv_data

csv_data = load_csvs('data/Investors_v2.csv')
len(csv_data)


599

In [22]:
csv_data[0]

Document(page_content='\ufeffInvestor Name: Frank M. Creer\nFund from Fund Master: Zone Ventures\nIndustry: Commercial Services,Software\nIndustry Copy: Software and Information Services\nVertical: Industrials,TMT\nInvestor Type: Venture Capital\nWebsite: www.zonevc.com\nPreferred Geography (from Fund from Fund Master): United States\nPreferred Investment Stage: Early Stage,Late Stage,PE Growth/Expansion,Seed\nPreferred Investment Type: Convertible Debt,SAFE,Debt\nMax Check size: 2.50\nMin check size: 1.00\nHQ Location: Menlo Park, CA\nURL (from Fund from Fund Master): www.zonevc.com\nPreferred TAM: Less than a Billion\nImpact Investor: No\n% Diversity in Managers/LPs: 25%\nAge of Fund/Investor Since (Years): 20.0\nNumber of investments (Portfolio Size): 20\nReturn Expectations: 10X and above\nExit Strategy Preference: IPO\nNetwork and Support Resources: Heavily Supportive\nDue diligence process: Heavily Involved\nDecision-making timeline: Months\nReputation: 4\nPortfolio Success Track

In [27]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

def split_docs(documents,chunk_size=1000,chunk_overlap=20):
  text_splitter = RecursiveCharacterTextSplitter(chunk_size=chunk_size, chunk_overlap=chunk_overlap)
  docs = text_splitter.split_documents(documents)
  return docs

docs = split_docs(csv_data)
print(len(docs))

921


In [28]:
csv_data = docs

In [7]:
docs[0]

Document(page_content='\ufeffInvestor Name: Frank M. Creer\nFund from Fund Master: Zone Ventures\nIndustry: Commercial Services,Software\nIndustry Copy: Software and Information Services\nVertical: Industrials,TMT\nInvestor Type: Venture Capital\nWebsite: www.zonevc.com\nPreferred Geography (from Fund from Fund Master): United States\nPreferred Investment Stage: Early Stage,Late Stage,PE Growth/Expansion,Seed\nPreferred Investment Type: Convertible Debt,SAFE,Debt\nMax Check size: 2.50\nMin check size: 1.00\nHQ Location: Menlo Park, CA\nURL (from Fund from Fund Master): www.zonevc.com\nPreferred TAM: Less than a Billion\nImpact Investor: No\n% Diversity in Managers/LPs: 25%\nAge of Fund/Investor Since (Years): 20.0\nNumber of investments (Portfolio Size): 20\nReturn Expectations: 10X and above\nExit Strategy Preference: IPO\nNetwork and Support Resources: Heavily Supportive\nDue diligence process: Heavily Involved\nDecision-making timeline: Months\nReputation: 4\nPortfolio Success Track

In [40]:
from langchain.embeddings import SentenceTransformerEmbeddings
embeddings = SentenceTransformerEmbeddings(model_name="all-MiniLM-L6-v2")

In [29]:
from langchain.vectorstores import Chroma
db = Chroma.from_documents(csv_data, embeddings)

In [15]:
query = "Name different fund masters that normally invest in healthcare?"
matching_docs = db.similarity_search(query)

matching_docs[1]


Document(page_content='\ufeffInvestor Name: Michael Y. Mashaal\nFund from Fund Master: HealthCor Partners Management\nIndustry: Healthcare Devices and Supplies,Healthcare Services,Pharmaceuticals and Biotechnology\nIndustry Copy: \nVertical: Digital Health,HealthTech,Life Sciences,Manufacturing\nInvestor Type: Growth/Expansion\nWebsite: www.healthcorpartners.com\nPreferred Geography (from Fund from Fund Master): United States\nPreferred Investment Stage: Early Stage,Late Stage,PE Growth/Expansion,Seed\nPreferred Investment Type: Convertible Debt,SAFE,Debt\nMax Check size: 40.00\nMin check size: 10.00\nHQ Location: New York, NY\nURL (from Fund from Fund Master): www.healthcorpartners.com\nPreferred TAM: \nImpact Investor: \n% Diversity in Managers/LPs: \nAge of Fund/Investor Since (Years): \nNumber of investments (Portfolio Size): \nReturn Expectations: \nExit Strategy Preference: \nNetwork and Support Resources: \nDue diligence process: \nDecision-making timeline: \nReputation: \nPortf

In [30]:
persist_directory = "chroma_db"

vectordb = Chroma.from_documents(
    documents=csv_data, embedding=embeddings, persist_directory=persist_directory
)

vectordb.persist()

In [31]:
import os
# Read API key from env file
os.environ["OPENAI_API_KEY"] = #os.getenv("OPENAI_KEY")


from langchain.chat_models import ChatOpenAI
from langchain.prompts.chat import (
    ChatPromptTemplate,
    SystemMessagePromptTemplate,
    AIMessagePromptTemplate,
    HumanMessagePromptTemplate,
)
from langchain.schema import AIMessage, HumanMessage, SystemMessage


model_name = "gpt-4"
llm = ChatOpenAI(model_name=model_name, temperature=0)

In [32]:
messages = [
    SystemMessage(
        content="You are a Venture capitalist who has 7+ years of experience in funding and investment domain."
    ),
    HumanMessage(
        content="Recommend at most 5 Fund Masters(from the investors data you have) for each founder info that is provided to you in python dictionary format"
    ),
]
llm(messages)

AIMessage(content="Sure, I can provide a Python function that would recommend 5 Fund Masters based on the founder's information. However, I don't have access to real investors data. I'll create a mock-up function that simulates this process.\n\n```python\ndef recommend_fund_masters(founder_info, investors_data):\n    # This function takes a founder's information and a list of investors data\n    # and returns a list of recommended Fund Masters.\n\n    # For the sake of this example, let's assume that the investors_data is a list of dictionaries,\n    # where each dictionary contains information about a single investor.\n\n    # Let's also assume that the founder_info is a dictionary containing information about the founder.\n\n    recommended_investors = []\n\n    # Here we would normally have some complex logic to match the founder with the best investors.\n    # This could be based on the founder's industry, the size of their company, their location, etc.\n    # For the sake of this 

In [33]:
template = (
    """You are a Venture capitalist consultant who has 7+ years of experience in funding and investment domain. You look into founders information and recommend investors name with respect to the data I have provided you using langchian csvreader.
    You receive a founder_info in python dictionary format and return recommded investors"""
)
system_message_prompt = SystemMessagePromptTemplate.from_template(template)
human_template = "{text}"
human_message_prompt = HumanMessagePromptTemplate.from_template(human_template)

In [34]:
import csv

founders_data = []
with open('data/founder_v2.csv', 'r') as file:
    reader = csv.reader(file)
    headers = next(reader)
    for row in reader:
        founders_data.append(dict(zip(headers, row)))

document = founders_data[10]

In [35]:
document

{'\ufeffName': 'Clair Marie McDade',
 'Company': 'Archneura Inc.',
 'Email': 'clairmarie@archneura.com',
 'Industry': 'Real Estate',
 'Verticals': 'Real Estate Technology,Construction Technology',
 'Target Customer': 'B2C',
 'Geography': 'United States of America (USA)',
 'Target Market (Locations)': 'United States of America (USA)',
 'Investment Stage': 'SEED',
 'Planned Raise Amount': '$1M to $2M',
 'Market size': '$5Billion to $10Billion',
 'Business Revenue Model': 'Retail',
 'Stage of development': 'SEED',
 'Revenue/profitability': 'Recurring',
 'Fundraising history': 'Friends and Family',
 'Traction': 'Certifications,Awards & recognition',
 'Team Key Roles & experience': 'Weak',
 'Intellectual property': 'Strong ',
 'Competitive landscape': 'No Competition',
 'Customer Base (# paying clients)': '90.0',
 'Growth potential': 'High',
 'Management experience': 'Startup Professional,First time founder'}

In [14]:
from langchain.chains import RetrievalQA

In [36]:

query = """
TASK: Recommend investors for a founder by matching features that a investor would consider while investing in a startup? \n
INSTRUCTION: While matching investors with founders, please keep in mind that investors always invest in startups that match
thier industry and verticals. eg: A founder who is developing a heathcare product, usually attract investors that invest 
in healthcare base industry and verticals. Please give importance to features like geography, TAM and other relevant features.
Please restrict yourself from recommending investors that don't invest in domain and vertical of a founder. \n
\nQUESTION: Recommend at most 5 Fund Masters(from the investors data you have) for this founder: {0}. \n """.format(document)
retrieval_chain = RetrievalQA.from_chain_type(llm, chain_type="stuff", retriever=db.as_retriever())
out=retrieval_chain.run(query)

In [37]:
out.split("\n")

["I'm sorry, but based on the information provided, none of the investors - Austin Walne, Albert Wenger, and Michael G. Borrus - invest in the Real Estate industry or in Real Estate Technology and Construction Technology verticals. Therefore, I cannot recommend any of these investors for Clair Marie McDade's startup, Archneura Inc."]

In [39]:

query = """
Can you see Watertower Ventures in the investors data I provided?\n """.format(document)
retrieval_chain = RetrievalQA.from_chain_type(llm, chain_type="stuff", retriever=db.as_retriever())
retrieval_chain.run(query)

'No, Watertower Ventures is not mentioned in the investors data you provided.'

In [ ]:
from langchain.chains.question_answering import load_qa_chain
chain = load_qa_chain(llm, chain_type="stuff",verbose=True)

query = """
ROLE: You are a Venture capatilist consultant who has 7+ years of experience in funding and investment domain. \n
TASK: Recommend investors for a founder by matching features that a investor would consider while investing in a startup? \n
INSTRUCTION: While matching investors with founders, please keep in mind that investors always invent in startups that match
thier industry and verticals. eg: A founder who is developing a heathcare product, usually attract investors that invest 
in healthcare base industry and verticals. Please give importance to features like geography, TAM and other relevant features.
Please restrict yourself from recommending investors that don't invest in domain and vertical of a founder. \n
QUESTION: Recommend at most 5 Fund Masters(from the investors data you have) for Sofia Yen who is the founder of PandiaHealth. 
        example for sophia yen, Shervin Pishevar(Fund maanger) would be a good option as he invests in Industires that work on healthcare."""
matching_docs = db.similarity_search(query)
answer =  chain.run(input_documents=matching_docs, question=query)
answer
